In [42]:
import numpy as np
from tqdm.auto import tqdm
from pyedflib import highlevel

from Datasets import *

## Cargamos los datos con las clases que ya tenemos creadas

In [2]:
dataset = EDFData("../Data/PSG1.edf", channels=['F4'])

e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:21: RuntimeWarning: Omitted 3 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(path)
e:\Python\TFM\SSD_IA3\Notebooks\Datasets.py:21: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_edf(path)


In [3]:
data = dataset.epochs.get_data()

In [4]:
data.shape

(882, 1, 15360)

## Como tenemos que obtener un array de forma `(canales, samples*epocas)`, lo más fácil es hacerlo a mano para asegurarnos de que no se nos lía nada al hacer `.reshape()`. Filtramos las épocas que exceden los límites físicos porque parece que se rompe un poco si los tocamos.

In [75]:
temp = None
skipped = 0
for i, a in tqdm(enumerate(data)):
    if (a*1e6 > 200).any() or (a*1e6 < -200).any():
        skipped += 1
        continue
    if temp is None:
        temp = a
    else:
        temp = np.concatenate([temp, a], -1)
    # if i==2:
    #     break
print(skipped)
a.shape, temp.shape

882it [00:13, 66.49it/s]

66


((1, 15360), (1, 12533760))

In [80]:
signals = temp*1e6
signals = np.vstack([signals,signals])
channel_names = ['F4', 'F4_2']

signal_headers = highlevel.make_signal_headers(channel_names, sample_rate=512, physical_min=-200, physical_max=200)

header = highlevel.make_header(patientname='patient_x', gender='Female')
header['annotations'] = [[start*30, 30, 'Patata'] for start in range(data.shape[0]-skipped)]
highlevel.write_edf('edf_file_nuestrosdatos.edf', signals=signals, signal_headers=signal_headers, header=header)

True